Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/training-with-deep-learning/distributed-pytorch-with-horovod/distributed-pytorch-with-horovod.png)

# Distributed PyTorch with Horovod
In this tutorial, you will train a PyTorch model on the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset using distributed training via [Horovod](https://github.com/uber/horovod) across a GPU cluster.

## Prerequisites
* If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, go through the [Configuration](../../../configuration.ipynb) notebook to install the Azure Machine Learning Python SDK and create an Azure ML `Workspace`
* Review the [tutorial](../train-hyperparameter-tune-deploy-with-pytorch/train-hyperparameter-tune-deploy-with-pytorch.ipynb) on single-node PyTorch training using Azure Machine Learning

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.33


## Diagnostics
Opt-in diagnostics for better experience, quality, and security of future releases.

In [2]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


## Initialize workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [3]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: contentRecommenderSystem
Azure region: eastus
Subscription id: 94ff7c1e-50c0-4466-a33b-232a0ccff39d
Resource group: proj_skMagic


## Create or attach existing AmlCompute
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. In this tutorial, we use Azure ML managed compute ([AmlCompute](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)) for our remote training compute resource. Specifically, the below code creates an `STANDARD_NC6` GPU cluster that autoscales from `0` to `4` nodes.

**Creation of AmlCompute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace, this code will skip the creation process.

As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute. 
print(compute_target.get_status().serialize())

Creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-06-24T06:12:03.529000+00:00', 'errors': None, 'creationTime': '2019-06-24T06:11:52.025872+00:00', 'modifiedTime': '2019-06-24T06:12:07.807585+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


The above code creates GPU compute. If you instead want to create CPU compute, provide a different VM size to the `vm_size` parameter, such as `STANDARD_D2_V2`.

## Train model on the remote compute
Now that we have the AmlCompute ready to go, let's run our distributed training job.

### Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script and any additional files your training script depends on.

In [5]:
import os

project_folder = './pytorch-distr-hvd'
os.makedirs(project_folder, exist_ok=True)

### Prepare training script
Now you will need to create your training script. In this tutorial, the script for distributed training of MNIST is already provided for you at `pytorch_horovod_mnist.py`. In practice, you should be able to take any custom PyTorch training script as is and run it with Azure ML without having to modify your code.

However, if you would like to use Azure ML's [metric logging](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#logging) capabilities, you will have to add a small amount of Azure ML logic inside your training script. In this example, at each logging interval, we will log the loss for that minibatch to our Azure ML run.

To do so, in `pytorch_horovod_mnist.py`, we will first access the Azure ML `Run` object within the script:
```Python
from azureml.core.run import Run
run = Run.get_context()
```
Later within the script, we log the loss metric to our run:
```Python
run.log('loss', loss.item())
```

Once your script is ready, copy the training script `pytorch_horovod_mnist.py` into the project directory.

In [7]:
import shutil

shutil.copy('pytorch_horovod_mnist.py', project_folder)

'./pytorch-distr-hvd/pytorch_horovod_mnist.py'

### Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed PyTorch tutorial. 

In [8]:
from azureml.core import Experiment

experiment_name = 'pytorch-distr-hvd'
experiment = Experiment(ws, name=experiment_name)

### Create a PyTorch estimator
The Azure ML SDK's PyTorch estimator enables you to easily submit PyTorch training jobs for both single-node and distributed runs. For more information on the PyTorch estimator, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-train-pytorch).

In [9]:
from azureml.core.runconfig import MpiConfiguration
from azureml.train.dnn import PyTorch

estimator = PyTorch(source_directory=project_folder,
                    compute_target=compute_target,
                    entry_script='pytorch_horovod_mnist.py',
                    node_count=2,
                    distributed_training=MpiConfiguration(),
                    use_gpu=True)

The above code specifies that we will run our training script on `2` nodes, with one worker per node. In order to execute a distributed run using MPI/Horovod, you must provide the argument `distributed_backend='mpi'`. Using this estimator with these settings, PyTorch, Horovod and their dependencies will be installed for you. However, if your script also uses other packages, make sure to install them via the `PyTorch` constructor's `pip_packages` or `conda_packages` parameters.

### Submit job
Run your experiment by submitting your estimator object. Note that this call is asynchronous.

In [10]:
run = experiment.submit(estimator)
print(run)

Run(Experiment: pytorch-distr-hvd,
Id: pytorch-distr-hvd_1561357026_bc1b43bb,
Type: azureml.scriptrun,
Status: Preparing)


### Monitor your run
You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes. You can see that the widget automatically plots and visualizes the loss metric that we logged to the Azure ML run.

In [11]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

Alternatively, you can block until the script has completed training before running more code.

In [12]:
run.wait_for_completion(show_output=True) # this provides a verbose log

RunId: pytorch-distr-hvd_1561357026_bc1b43bb

Streaming azureml-logs/20_image_build_log.txt

2019/06/24 06:17:11 Downloading source code...
2019/06/24 06:17:12 Finished downloading source code
2019/06/24 06:17:12 Using acb_vol_cdae5ca7-4ef2-4f63-b8c4-ef26c092edad as the home volume
2019/06/24 06:17:12 Creating Docker network: acb_default_network, driver: 'bridge'
2019/06/24 06:17:12 Successfully set up Docker network: acb_default_network
2019/06/24 06:17:12 Setting up Docker configuration...
2019/06/24 06:17:13 Successfully set up Docker configuration
2019/06/24 06:17:13 Logging in to registry: contentrecom1004ddbb.azurecr.io
2019/06/24 06:17:14 Successfully logged into contentrecom1004ddbb.azurecr.io
2019/06/24 06:17:14 Executing step ID: acb_step_0. Timeout(sec): 1800, Working directory: '', Network: 'acb_default_network'
2019/06/24 06:17:14 Scanning for dependencies...
2019/06/24 06:17:15 Successfully scanned dependencies
2019/06/24 06:17:15 Launching container with name: acb_step_0

  ERROR: Complete output from command /azureml-envs/azureml_34c021840e4cd4edf5cce81c8c92bc3e/bin/python -u -c 'import setuptools, tokenize;__file__='"'"'/tmp/pip-install-ex8kpgom/horovod/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-8cyizfmk --python-tag cp36:


  ERROR: warning: no previously-included files found matching 'setup.pyc'
  zip_safe flag not set; analyzing archive contents...
  pycparser.ply.__pycache__.lex.cpython-36: module references __file__
  pycparser.ply.__pycache__.lex.cpython-36: module MAY be using inspect.getsourcefile
  pycparser.ply.__pycache__.yacc.cpython-36: module references __file__
  pycparser.ply.__pycache__.yacc.cpython-36: module MAY be using inspect.getsourcefile
  pycparser.ply.__pycache__.yacc.cpython-36: module MAY be using inspect.stack
  pycparser.ply.__pycache__.ygen.cpython-36: module references __file__
  
  Installed /tmp/pip-install-ex8kpgom/horovod/.eggs/pycparser-2.19-py3.6.egg
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.6
  creating build/lib.linux-x86_64-3.6/horovod
  copying horovod/__init__.py -> build/lib.linux-x86_64-3.6/horovod
  creating build/lib.linux-x86_64-3.6/horovod/common
  copying horovod/common/__init__.py -> build

  Running setup.py install for horovod: started
    Running setup.py install for horovod: finished with status 'done'
#
# To activate this environment, use:
# > source activate /azureml-envs/azureml_34c021840e4cd4edf5cce81c8c92bc3e
#
# To deactivate an active environment, use:
# > source deactivate
#


Removing intermediate container 0c09d19496f6
 ---> ae05b118dfe6
Step 9/14 : ENV PATH /azureml-envs/azureml_34c021840e4cd4edf5cce81c8c92bc3e/bin:$PATH
 ---> Running in ce0964503583
Removing intermediate container ce0964503583
 ---> fc8cab448864
Step 10/14 : ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/azureml_34c021840e4cd4edf5cce81c8c92bc3e
 ---> Running in af245f917a6b
Removing intermediate container af245f917a6b
 ---> 485e2c5894a1
Step 11/14 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_34c021840e4cd4edf5cce81c8c92bc3e/lib:$LD_LIBRARY_PATH
 ---> Running in fd66a7427a21
Removing intermediate container fd66a7427a21
 ---> 69b518203cd1
Step 12/14 : COPY azureml-setup/spark_cache.py azure

Train Epoch: 2 [4480/30000 (15%)]	Loss: 0.496823
Train Epoch: 2 [5120/30000 (17%)]	Loss: 0.417140
Train Epoch: 2 [5760/30000 (19%)]	Loss: 0.357129
Train Epoch: 2 [6400/30000 (21%)]	Loss: 0.554394
Train Epoch: 2 [7040/30000 (23%)]	Loss: 0.350656
Train Epoch: 2 [7680/30000 (26%)]	Loss: 0.425094
Train Epoch: 2 [8320/30000 (28%)]	Loss: 0.530909
Train Epoch: 2 [8960/30000 (30%)]	Loss: 0.376844
Train Epoch: 2 [9600/30000 (32%)]	Loss: 0.461306
Train Epoch: 2 [10240/30000 (34%)]	Loss: 0.854818
Train Epoch: 2 [10880/30000 (36%)]	Loss: 0.834800
Train Epoch: 2 [11520/30000 (38%)]	Loss: 0.359380
Train Epoch: 2 [12160/30000 (41%)]	Loss: 0.308885
Train Epoch: 2 [12800/30000 (43%)]	Loss: 0.363920
Train Epoch: 2 [13440/30000 (45%)]	Loss: 0.622393
Train Epoch: 2 [14080/30000 (47%)]	Loss: 0.334052
Train Epoch: 2 [14720/30000 (49%)]	Loss: 0.460806
Train Epoch: 2 [15360/30000 (51%)]	Loss: 0.555016
Train Epoch: 2 [16000/30000 (53%)]	Loss: 0.543872
Train Epoch: 2 [16640/30000 (55%)]	Loss: 0.412329
Train Epo

Train Epoch: 5 [18560/30000 (62%)]	Loss: 0.253278
Train Epoch: 5 [19200/30000 (64%)]	Loss: 0.449022
Train Epoch: 5 [19840/30000 (66%)]	Loss: 0.256291
Train Epoch: 5 [20480/30000 (68%)]	Loss: 0.349666
Train Epoch: 5 [21120/30000 (70%)]	Loss: 0.232925
Train Epoch: 5 [21760/30000 (72%)]	Loss: 0.079942
Train Epoch: 5 [22400/30000 (75%)]	Loss: 0.130891
Train Epoch: 5 [23040/30000 (77%)]	Loss: 0.134720
Train Epoch: 5 [23680/30000 (79%)]	Loss: 0.138159
Train Epoch: 5 [24320/30000 (81%)]	Loss: 0.160537
Train Epoch: 5 [24960/30000 (83%)]	Loss: 0.192191
Train Epoch: 5 [25600/30000 (85%)]	Loss: 0.119760
Train Epoch: 5 [26240/30000 (87%)]	Loss: 0.078714
Train Epoch: 5 [26880/30000 (90%)]	Loss: 0.142437
Train Epoch: 5 [27520/30000 (92%)]	Loss: 0.132769
Train Epoch: 5 [28160/30000 (94%)]	Loss: 0.117062
Train Epoch: 5 [28800/30000 (96%)]	Loss: 0.171121
Train Epoch: 5 [29440/30000 (98%)]	Loss: 0.235268

Test set: Average loss: 0.0790, Accuracy: 97.59%

Train Epoch: 6 [0/30000 (0%)]	Loss: 0.147479
Trai

Train Epoch: 9 [5760/30000 (19%)]	Loss: 0.113768
Train Epoch: 9 [6400/30000 (21%)]	Loss: 0.180187
Train Epoch: 9 [7040/30000 (23%)]	Loss: 0.245247
Train Epoch: 9 [7680/30000 (26%)]	Loss: 0.154771
Train Epoch: 9 [8320/30000 (28%)]	Loss: 0.132401
Train Epoch: 9 [8960/30000 (30%)]	Loss: 0.201390
Train Epoch: 9 [9600/30000 (32%)]	Loss: 0.112674
Train Epoch: 9 [10240/30000 (34%)]	Loss: 0.104815
Train Epoch: 9 [10880/30000 (36%)]	Loss: 0.300736
Train Epoch: 9 [11520/30000 (38%)]	Loss: 0.212344
Train Epoch: 9 [12160/30000 (41%)]	Loss: 0.117101
Train Epoch: 9 [12800/30000 (43%)]	Loss: 0.184125
Train Epoch: 9 [13440/30000 (45%)]	Loss: 0.132576
Train Epoch: 9 [14080/30000 (47%)]	Loss: 0.188562
Train Epoch: 9 [14720/30000 (49%)]	Loss: 0.442276
Train Epoch: 9 [15360/30000 (51%)]	Loss: 0.235592
Train Epoch: 9 [16000/30000 (53%)]	Loss: 0.176223
Train Epoch: 9 [16640/30000 (55%)]	Loss: 0.238819
Train Epoch: 9 [17280/30000 (58%)]	Loss: 0.209362
Train Epoch: 9 [17920/30000 (60%)]	Loss: 0.110100
Train E

{'runId': 'pytorch-distr-hvd_1561357026_bc1b43bb',
 'target': 'gpucluster',
 'status': 'Completed',
 'startTimeUtc': '2019-06-24T06:30:09.179072Z',
 'endTimeUtc': '2019-06-24T06:34:06.571799Z',
 'properties': {'azureml.runsource': 'experiment',
  'AzureML.DerivedImageName': 'azureml/azureml_595dfaaa4e880ada400f8d4ed7eada57',
  'ContentSnapshotId': '7fa97da6-1a82-487b-b70e-e9b54671b43e',
  'azureml.git.repository_uri': None,
  'azureml.git.branch': None,
  'azureml.git.commit': None,
  'azureml.git.dirty': 'False',
  'azureml.git.build_id': None,
  'azureml.git.build_uri': None,
  'mlflow.source.git.branch': None,
  'mlflow.source.git.commit': None,
  'mlflow.source.git.repoURL': None},
 'runDefinition': {'script': 'pytorch_horovod_mnist.py',
  'arguments': [],
  'sourceDirectoryDataStore': 'workspaceblobstore',
  'framework': 'Python',
  'communicator': 'Mpi',
  'target': 'gpucluster',
  'dataReferences': {'workspaceblobstore': {'dataStoreName': 'workspaceblobstore',
    'mode': 'Mount